# Simple 3D Finite Plasticity model



In [1]:
import numpy as np

import dolfinx

from mpi4py import MPI
from petsc4py import PETSc

from dolfinx import fem, mesh, io, plot, log,default_scalar_type
from dolfinx.fem import (Constant, dirichletbc, Function, functionspace, Expression )
from dolfinx.fem.petsc import NonlinearProblem,LinearProblem
from dolfinx.nls.petsc import NewtonSolver
from dolfinx.io import XDMFFile
import ufl
from ufl import (TestFunctions, TrialFunction, Identity, grad, det, div, dev, inv, tr, sqrt, conditional , gt, dx, inner, derivative, dot, ln, split,TestFunction,indices,as_tensor)
from basix.ufl import element, mixed_element, quadrature_element
from datetime import datetime
from dolfinx.plot import vtk_mesh

import pyvista
pyvista.set_jupyter_backend('client')
## Define temporal parameters

## Some parameters

In [2]:
e_dot = 0.001 # strain rate per second 
e_max = 0.01 # absolute value of maximum strain


K_val = 160e9 # Bulk modulus 
G_val = 80e9 # Shear modulus
Y0 = 100e6 # Initial yield 
h0 = 0 # Initial hardening modulus 
Y_star = 100e6 # Yield saturation 
r = 0 #Hardening exponent 
C_back = 0 #Backstress modulus 
gamma = 0 # Nonlinear kinematic parameter



# Define Geometry

In [3]:
length = 10.0 # mm
longside= 100
num_elements = 10
domain = mesh.create_box(MPI.COMM_WORLD,[[0.0,0.0,0.0],[longside,length,length]],[num_elements*10,num_elements,num_elements],mesh.CellType.tetrahedron)

In [4]:
plotter = pyvista.Plotter()
vtkdata = vtk_mesh(domain, domain.topology.dim)
grid = pyvista.UnstructuredGrid(*vtkdata)
actor = plotter.add_mesh(grid, show_edges=True)
plotter.show()
plotter.close()

Widget(value="<iframe src='http://localhost:41537/index.html?ui=P_0xffff7d62a320_0&reconnect=auto' style='widt…

# Defining Function Spaces

In [5]:
# Tensor space
T_Ele = quadrature_element(domain.topology.cell_name(),value_shape=(3,3),degree=1)

n = 0
U_Ele = element("Lagrange", domain.basix_cell(),1,shape=(3,))

#T_quad = functionspace(domain,T_Ele)
V_u = functionspace(domain,U_Ele)

In [6]:
#MAT Properties

K = Constant(domain,K_val) #200 MPa Bulk Modulus
G = Constant(domain,G_val) #80 Mpa Shear Modulus

I = Identity(3)
II = ufl.outer(I,I)
i,j,k,l = indices(4)
Ident_sym_ii =1/2 * (I[i,k]*I[j,l] + I[i,l]*I[j,k])

Ident_sym = as_tensor(Ident_sym_ii,(i,j,k,l))
C = 2*G*Ident_sym + (K-(2/3)*G)*II

In [ ]:
u = Function(V_u)

#Total Strain
E = ufl.sym(grad(u))

Ep = En - Epn